In [22]:
from finlab.data import Data
from collections import defaultdict

global allstocklist  
allstocklist = []
稅後淨利 = Data().get('本期淨利（淨損）', 1)
for i in 稅後淨利.keys():
    allstocklist.append(i)

def Buffett(data):
    
    #此策略僅兩個條件:  1.近三年平均ROE>.15  2.最近年度股東權益報酬率＞平均值(市場及產業)
    稅後淨利 = data.get('本期淨利（淨損）', 12)
    
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 12)
    權益總額 = data.get('權益總額', 12)
    
    # 並且把它們合併起來    
    權益總計.fillna(權益總額, inplace=True)
    
    #條件1.ROE近三年平均
    近三年平均ROE = (((稅後淨利.iloc[-1]+稅後淨利.iloc[-2]+稅後淨利.iloc[-3]+稅後淨利.iloc[-4])/權益總計.iloc[-1])+((稅後淨利.iloc[-5]+稅後淨利.iloc[-6]+稅後淨利.iloc[-7]+稅後淨利.iloc[-8])/權益總計.iloc[-5])+((稅後淨利.iloc[-9]+稅後淨利.iloc[-10]+稅後淨利.iloc[-11]+稅後淨利.iloc[-12])/權益總計.iloc[-9]))/3
    
    condition1 = 近三年平均ROE > 0.15
    
    #存取好condition1所抓出來的股票
    results = condition1[condition1]
    
    #找出與condition1所選出的同類股
    stockindex = []
    for i in range(len(results)):
        stockindex.append(int(results.index[i]))
    
    #存取所有同類股代碼，一對多的dict
    samekind = defaultdict(list)
    for j in range(len(stockindex)):
        for i in range(len(allstocklist)):
            if(str(int(allstocklist[i])//100)==str(int(stockindex[j])//100)):
                samekind[stockindex[j]].append(allstocklist[i])
    
    #計算同類股(同行)的平均ROE，方法同上一對多的dict，方便後續比較個股與同行業平均
    totalni = defaultdict(list)
    for i in samekind.keys():
        sumroe = 0
        avgroe = 0
        for j in samekind[i]:
            sumroe = (稅後淨利[j].iloc[-1]/權益總計[j].iloc[-1])+sumroe
        avgroe = sumroe/len(samekind[i])
        totalni[i].append(avgroe)
    
    #跟同業比較後，條件符合者存入final變數最後回傳
    final = []    
    for i in range(len(stockindex)):
        if (稅後淨利[str(stockindex[i])].iloc[-1]/權益總計[str(stockindex[i])].iloc[-1])>totalni[stockindex[i]]:
            final.append(stockindex[i])
    return final 
 

In [1]:
import strategies.master
from finlab.data import Data
from strategies import master
data = Data()
res1 = master.Buffett(data)
res1

[1109,
 1215,
 1216,
 1227,
 1231,
 1232,
 1256,
 1264,
 1268,
 1325,
 1476,
 1477,
 1515,
 1527,
 1537,
 1541,
 1558,
 1565,
 1580,
 1582,
 1590,
 1597,
 1702,
 1707,
 1730,
 1773,
 1788,
 1808,
 2028,
 2063,
 2108,
 2114,
 2207,
 2227,
 2327,
 2330,
 2337,
 2345,
 2348,
 2360,
 2373,
 2377,
 2379,
 2383,
 2385,
 2395,
 2404,
 2420,
 2421,
 2428,
 2439,
 2455,
 2458,
 2464,
 2467,
 2480,
 2492,
 2546,
 2614,
 2640,
 2643,
 2707,
 2729,
 2912,
 2915,
 2926,
 2939,
 3008,
 3023,
 3026,
 3030,
 3034,
 3043,
 3044,
 3045,
 3088,
 3090,
 3092,
 3105,
 3130,
 3147,
 3152,
 3167,
 3169,
 3213,
 3217,
 3226,
 3293,
 3324,
 3373,
 3402,
 3406,
 3413,
 3443,
 3526,
 3529,
 3533,
 3537,
 3541,
 3546,
 3556,
 3558,
 3563,
 3570,
 3580,
 3611,
 3665,
 3708,
 4105,
 4107,
 4119,
 4123,
 4137,
 4163,
 4190,
 4205,
 4506,
 4549,
 4551,
 4721,
 4735,
 4736,
 4754,
 4763,
 4912,
 4953,
 4966,
 4968,
 4977,
 5209,
 5212,
 5220,
 5234,
 5269,
 5274,
 5278,
 5287,
 5288,
 5289,
 5299,
 5305,
 5306,
 5347,

In [101]:
#班傑明．葛拉漢
#1. 年營業額大於市場平均值的公司
#2. 過去5年皆有盈餘的公司
#3. 連續2年皆支付現金股利的公司
#4. 流動比例>200%.....done
#5. 流動淨資產 – 長期負債>0.....done
#6. (近三年平均稅後淨利-近5年平均稅後淨利)/近5年平均稅後淨利的絕對值>0.33
#7. PER1(以近3年平均每股盈餘計算)<= 近三年PER1平均
#8. PER(以近4季每股盈餘計算)*PBR <= 近三年PER*PBR平均
from finlab.data import Data
from collections import defaultdict

def Benjamin(data):
    稅後淨利 = data.get('本期淨利（淨損）', 12)
    權益總計 = data.get('權益總計', 12)
    權益總額 = data.get('權益總額', 12) 
    權益總計.fillna(權益總額, inplace=True)
    
    #condition1 4
    ca = data.get('流動資產合計',1)
    cl = data.get('流動負債合計',1)
    流動比率 = ca/cl
    condition1 = 流動比率 > 2
    
    #condition2 5.改用ca-長期負債(非流動負債)
    ld = data.get('非流動負債合計',1)
    可動用金 = ca-ld
    condition2 = 可動用金 > 0
    
    #condition3 2.改兩年 因為有些資料為nan 
    #condition3 = int(稅後淨利.iloc[-1]) > 0 
    
    #select_stock = condition1 & condition2 & condition3 
    
    print(select_stock[select_stock])

In [102]:
data = Data()
res = Benjamin(data)
res

TypeError: cannot convert the series to <class 'int'>

In [104]:
div = data.get('股利',1)

Data: **ERROR: cannot find 股利 in database
